# TensorFlow Tutorial - 4. MLP ( Multi Layer Perceptron )

본 문서는 TensorFlow 를 사용하여 Deep Learning을 구현하기 위한 기초적인 실습 자료이다.

The code and comments are written by Dong-Hyun Kwak <imcomking@gmail.com><br>
Upgraed to Tensorflow v1.9 by NamJungGu <nowage@gmail.com> 

<a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by-nc-sa/4.0/88x31.png" /></a><br />This work is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/">Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International License</a>.





In [3]:
import tensorflow as tf
import numpy as np
tf.reset_default_graph()
tf.__version__

'1.15.0'

In [4]:
#[sepal length, sepal width, petal length, petal width]
train_X=np.genfromtxt('iris_data_train.csv', delimiter=',')
# [setosa, versicolor, virginica]
train_y=np.genfromtxt('iris_labels_train.csv', delimiter=',')

test_X=np.genfromtxt('iris_data_test.csv', delimiter=',')
test_y=np.genfromtxt('iris_labels_test.csv', delimiter=',')

#데이터 shape 을 확인해 봅니다.
print(train_X.shape,train_y.shape)

(100, 4) (100, 3)


In [5]:
# placeholder is used for feeding data.
x = tf.placeholder("float", shape=[None, 4], name = 'x') # none represents variable length of dimension. 784 is the dimension of MNIST data.
y_target = tf.placeholder("float", shape=[None, 3], name = 'y_target') # shape argument is optional, but this is useful to debug.


# all the variables are allocated in GPU memory
W1 = tf.Variable(tf.zeros([4, 5]), name = 'W1')   # create (4 * 5) matrix
b1 = tf.Variable(tf.zeros([5]), name = 'b1')        # create (1 * 5) vector
h1 = tf.sigmoid(tf.matmul(x, W1) + b1, name = 'h1')   # compute --> sigmoid(weighted summation)

# all the variables are allocated in GPU memory
W2 = tf.Variable(tf.zeros([5, 6]), name = 'W2')   # create (5 * 6) matrix
b2 = tf.Variable(tf.zeros([6]), name = 'b2')        # create (1 * 6) vector
h2 = tf.sigmoid(tf.matmul(h1, W2) + b2, name = 'h2')   # compute --> sigmoid(weighted summation)


# Repeat again
W3 = tf.Variable(tf.zeros([6, 3]), name = 'W3')     # create (6 * 3) matrix
b3 = tf.Variable(tf.zeros([3]), name = 'b3')          # create (1 * 3) vector
y = tf.nn.softmax(tf.matmul(h2, W3) + b3, name = 'y')  # compute classification --> softmax(weighted summation)

# 최종결과는 ( 투입값 * 3) matrix

# define the Loss function
cross_entropy = -tf.reduce_sum(y_target*tf.log(y), name = 'cross_entropy')


# define optimization algorithm
train_step = tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)



correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_target, 1))
# correct_prediction is list of boolean which is the result of comparing(model prediction , data)


accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float")) 
# tf.cast() : changes true -> 1 / false -> 0
# tf.reduce_mean() : calculate the mean

In [6]:
# Create Session
sess = tf.Session(config=tf.ConfigProto(gpu_options=tf.GPUOptions(allow_growth=True))) # open a session which is a envrionment of computation graph.
sess.run(tf.global_variables_initializer())# initialize the variables

# training the MLP
for i in range(10001): # minibatch iteraction
    #batch = mnist.train.next_batch(100) # minibatch size
    sess.run(train_step, feed_dict={x: train_X, y_target: train_y}) # placeholder's none length is replaced by i:i+100 indexes

    if i%1000 == 0:
        train_accuracy = sess.run(accuracy, feed_dict={x:train_X, y_target: train_y})
        print ("step %d, training accuracy: %.3f"%(i, train_accuracy))

# for given x, y_target data set
print  ("test accuracy: %g"% sess.run(accuracy, feed_dict={x: test_X, y_target: test_y}))
#sess.close()

step 0, training accuracy: 0.350
step 1000, training accuracy: 0.350
step 2000, training accuracy: 0.350
step 3000, training accuracy: 0.350
step 4000, training accuracy: 0.660
step 5000, training accuracy: 0.870
step 6000, training accuracy: 0.700
step 7000, training accuracy: 0.880
step 8000, training accuracy: 0.870
step 9000, training accuracy: 0.910
step 10000, training accuracy: 0.920
test accuracy: 0.92
